In [1]:
import numpy as np
import h5py
import matplotlib.pyplot as plt
from testCases_v2 import *
from dnn_utils_v2 import sigmoid, sigmoid_backward, relu, relu_backward

***2 Layer Neural Network***

In [3]:
def initialize_parameters(n_x, n_h, n_y):

    np.random.seed(1)

    W1= np.random.randn(n_h, n_x) * 0.01
    b1= np.zeros(shape=(n_h, 1))
    W2= np.random.randn(n_y, n_h) * 0.01
    b2= np.zeros(shape=(n_y,1))

    assert(W1.shape == (n_h, n_x))
    assert(b1.shape == (n_h, 1))
    assert(W2.shape == (n_y, n_h))
    assert(b2.shape == (n_y, 1))

    parameters = {
        "W1": W1, 
        "b1" : b1, 
        "W2" : W2, 
        "b2" : b2, 
        
    }
    
    return parameters

In [4]:
parameters = initialize_parameters(2,2, 1)
print("W1 is:" + str(parameters['W1']))
print("b1 is:" + str(parameters['b2']))
print("W2 is:" + str(parameters['W2']))
print("b2 is:" + str(parameters['b2']))

W1 is:[[ 0.01624345 -0.00611756]
 [-0.00528172 -0.01072969]]
b1 is:[[0.]]
W2 is:[[ 0.00865408 -0.02301539]]
b2 is:[[0.]]


In [5]:
def initialize_parameters_deep(layer_dims):

    np.random.seed(3)
    parameters = {}
    L= len(layer_dims)

    for l in range(1,L):
        parameters['W' + str(l)] = np.random.randn(layer_dims[l], layer_dims[l-1]) *  0.01
        parameters['b' + str(l)] = np.zeros((layer_dims[l], 1)) 

    assert(parameters['W' + str(l)].shape == (layer_dims[l], layer_dims[l-1]))
    assert(parameters['b' + str(l)].shape == (layer_dims[l], 1))
    
   
    return parameters

In [6]:
parameters = initialize_parameters_deep([5, 4, 3])
print("W1 = " + str(parameters["W1"]))
print("b1 = " + str(parameters["b1"]))
print("W2 = " + str(parameters["W2"]))
print("b2 = " + str(parameters["b2"]))

W1 = [[ 0.01788628  0.0043651   0.00096497 -0.01863493 -0.00277388]
 [-0.00354759 -0.00082741 -0.00627001 -0.00043818 -0.00477218]
 [-0.01313865  0.00884622  0.00881318  0.01709573  0.00050034]
 [-0.00404677 -0.0054536  -0.01546477  0.00982367 -0.01101068]]
b1 = [[0.]
 [0.]
 [0.]
 [0.]]
W2 = [[-0.01185047 -0.0020565   0.01486148  0.00236716]
 [-0.01023785 -0.00712993  0.00625245 -0.00160513]
 [-0.00768836 -0.00230031  0.00745056  0.01976111]]
b2 = [[0.]
 [0.]
 [0.]]


In [7]:
def linear_forward(A, W, b):

    Z = np.dot(W, A) + b

    assert(Z.shape == (W.shape[0], A.shape[1]) )
    cache = (A, W,b )
    return Z, cache
 

In [8]:
A, W, b = linear_forward_test_case()

Z, linear_cache = linear_forward(A,W,b)
print("Z =" + str(Z))


Z =[[ 3.26295337 -1.23429987]]


In [9]:
def linear_activation_forward(A_prev, W, b, activation):

    if activation == "sigmoid":
        Z, cache = linear_forward(A_prev, W, b)
        A, activation_cache= sigmoid(Z)

    elif activation == "relu":
        Z, cache = linear_forward(A_prev, W, b)
        A, activation_cache= relu(Z)

    assert(A.shape == (W.shape[0], A_prev.shape[1]))
    cache = (linear_cache, activation_cache)
        
    
    return A, activation_cache

In [10]:
A_prev, W, b = linear_activation_forward_test_case()

A, activation_cache = linear_activation_forward(A_prev, W, b, activation= "sigmoid")
print("With Sigmoid: A= " + str(A))

A, activation_cache = linear_activation_forward(A_prev, W, b, activation= "relu")
print("With Relu: A= " + str(A))

With Sigmoid: A= [[0.96890023 0.11013289]]
With Relu: A= [[3.43896131 0.        ]]


In [11]:
def L_model_forward(X, parameters):
    caches = []
    A= X
    L= len(parameters) // 2

    for l in range(1, L):
        A_prev = A

    A, cache = linear_activation_forward(A_prev,
                                         parameters['W' + str(l)], 
                                         parameters['b' + str(l)],
                                         activation= 'relu')
    caches.append(cache)

    AL, cache = linear_activation_forward(A, 
                                         parameters['W' + str(L)],
                                         parameters['b' + str(L)],
                                          activation = 'sigmoid')
    caches.append(cache)

    assert(AL.shape == (1, X.shape[1]))
    
    return AL, caches

In [12]:
X, parameters = L_model_forward_test_case()
AL, caches = L_model_forward(X, parameters)

print("AL = " + str(AL))
print("Length of caches list = " + str(len(caches)))

AL = [[0.17007265 0.2524272 ]]
Length of caches list = 2


In [29]:
def compute_cost(AL, Y):

    m = Y.shape[1]
    eps= 1e-8
    AL = np.clip(AL, eps, 1-eps)
    cost = (-1/m)*np.sum( np.multiply(Y, np.log(AL) ) + np.multiply( 1-Y, np.log(1-AL) ) )

    cost= np.squeeze(cost)
    assert(cost.shape ==  () )

    return cost

In [31]:
Y, AL = compute_cost_test_case()
cost = compute_cost(AL, Y)
print("Cost = " + str(cost))

Cost = 0.41493159961539694


In [35]:
def linear_backward(dZ, cache):

    A_prev, W, b = cache
    m = A_prev.shape[1]

    dW = np.dot(dZ, cache[0].T)/m
    db = np.squeeze( np.sum(dZ, axis= 1, keepdims= True))/ m
    dA_prev = np.dot(cache[1].T, dZ)/m

    assert(dA_prev.shape == A_prev.shape)
    assert(dW.shape == W.shape)
    assert(isinstance(db, float))

    return dA_prev, dW, db
    

In [37]:
dZ, cache = linear_backward_test_case()

dA_prev, dW, db = linear_backward(dZ, cache)

print("dA_prev = " + str(dA_prev))
print("dW = " + str(dW))
print("db = " + str(db))

dA_prev = [[ 0.25911484 -0.09758711]
 [-0.20253181  0.07627696]
 [ 1.18748412 -0.44722695]]
dW = [[-0.10076895  1.40685096  1.64992505]]
db = 0.5062944750065832
